In [122]:
import json
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

# Filenames

In [123]:
files_path = r"D:\thesis_code\datasets\monado_slam\MOO07_mapping_easy\mav0\cam0\data.csv"
files = pd.read_csv(files_path, header=0, names=('timestamp', 'filename'))

files['timestamp'] = pd.to_datetime(files['timestamp'], unit='ns')
files = files.sort_values(by='timestamp').reset_index(drop=True)

# files['filename'] = files['filename'].str.replace(".png", "", regex=False)

files.head()

,timestamp,filename
0,1970-01-01 02:24:17.516925124,8657519072990.png
1,1970-01-01 02:24:17.550229924,8657552377790.png
2,1970-01-01 02:24:17.583504024,8657585651890.png
3,1970-01-01 02:24:17.616793524,8657618941390.png
4,1970-01-01 02:24:17.650083124,8657652230990.png


# Ground Truth

In [124]:
gt_path = r'D:\thesis_code\datasets\monado_slam\MOO07_mapping_easy\mav0\gt\data.csv'

gt = pd.read_csv(
    gt_path, 
     header=0, 
     names=('timestamp', 'px', 'py', 'pz', 'qw', 'qx', 'qy', 'qz')
)

gt['timestamp'] = pd.to_datetime(gt['timestamp'], unit='ns')
gt = gt.sort_values(by='timestamp').reset_index(drop=True)

gt.head()

,timestamp,px,py,pz,qw,qx,qy,qz
0,1970-01-01 02:24:17.494950908,1.69970,-1.18114,-0.880342,-0.044995,0.937637,-0.003759,-0.344672
1,1970-01-01 02:24:17.496016256,1.69970,-1.18114,-0.880342,-0.044996,0.937637,-0.003759,-0.344672
2,1970-01-01 02:24:17.497081665,1.69970,-1.18114,-0.880342,-0.044996,0.937637,-0.003759,-0.344671
3,1970-01-01 02:24:17.498147354,1.69971,-1.18110,-0.880340,-0.044982,0.937635,-0.003775,-0.344679
4,1970-01-01 02:24:17.499211132,1.69971,-1.18110,-0.880340,-0.044981,0.937635,-0.003775,-0.344679


# Align Filename Row to Latest Ground Truth Row

In [125]:
aligned_df = pd.merge_asof(
    files, gt,
    on="timestamp",
    direction="backward",
).dropna()
            
aligned_df.head()

,timestamp,filename,px,py,pz,qw,qx,qy,qz
0,1970-01-01 02:24:17.516925124,8657519072990.png,1.69969,-1.18112,-0.880367,-0.044970,0.937634,-0.003780,-0.344683
1,1970-01-01 02:24:17.550229924,8657552377790.png,1.69974,-1.18114,-0.880359,-0.044961,0.937628,-0.003784,-0.344699
2,1970-01-01 02:24:17.583504024,8657585651890.png,1.69970,-1.18115,-0.880272,-0.044974,0.937616,-0.003818,-0.344731
3,1970-01-01 02:24:17.616793524,8657618941390.png,1.69972,-1.18115,-0.880272,-0.044915,0.937620,-0.003811,-0.344728
4,1970-01-01 02:24:17.650083124,8657652230990.png,1.69972,-1.18117,-0.880289,-0.044902,0.937620,-0.003820,-0.344728


# Setting T_i_c0

In [126]:
calib_path = r"D:/thesis_code/datasets/monado_slam/M_monado_datasets_MO_odyssey_plus_extras_calibration.json"

In [127]:
with open(calib_path, 'r') as file:
    data = json.load(file) 
    data = data['value0']['T_imu_cam']

In [128]:
def to_transformation_matrix(d):
    px, py, pz = d['px'], d['py'], d['pz']
    qx, qy, qz, qw = d['qx'], d['qy'], d['qz'], d['qw']
    
    rotation = R.from_quat([qx, qy, qz, qw]).as_matrix()
    
    # 4x4 transformation matrix
    transformation = np.eye(4)  # Identity matrix
    transformation[:3, :3] = rotation  # Set rotation
    transformation[:3, 3] = [px, py, pz]  # Set translation
    
    return transformation

In [129]:
transformation_matrices = [to_transformation_matrix(d) for d in data]

T_i_c0 = transformation_matrices[0]
T_i_c1 = transformation_matrices[1]

T_i_c0

array([[ 0.9184477 ,  0.0068145 , -0.39548373, -0.08323389],
       [ 0.13570781,  0.93373322,  0.33124865, -0.00221837],
       [ 0.37153359, -0.35790479,  0.85666035,  0.00562664],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

# Find T_w_c0 for 2 timesteps

In [130]:
transformation_matrices = [
    to_transformation_matrix(entry[1]) 
    for entry in aligned_df.iterrows()
]

In [135]:
T_w_i = transformation_matrices[0]
T_w_c0_t1 = T_w_i @ T_i_c0

T_w_i = transformation_matrices[500]
T_w_c0_t2 = T_w_i @ T_i_c0

# Find Angle between them

In [136]:
R_t1 = T_w_c0_t1[:3, :3]
R_t2 = T_w_c0_t2[:3, :3]

R_relative = np.linalg.inv(R_t1) @ R_t2

euler_angles = R.from_matrix(R_relative).as_euler('xyz', degrees=True) 

print("Angles : ", euler_angles)

Angles :  [-3.52622663 39.82024626 25.43582763]


In [137]:
np.linalg.norm(euler_angles)

47.3821444603981